# Problem 9: SQL Operations

This problem will test your ability to manipulate two simple SQL tables. You may find a problem easier to complete using Pandas, or you may find a problem easier to complete in SQL. We will provide you will a SQLite database containing two tables, and two Pandas Dataframes that are identical to the SQLite tables. 

In [1]:
import sys
import pandas as pd
import sqlite3 as db
from IPython.display import display

def get_data_path(filebase):
    return f"resource/asnlib/publicdata/movies/{filebase}"

print(f"* Python version: {sys.version}")
print(f"* pandas version: {pd.__version__}")
print(f"* sqlite3 version: {db.version}")

* Python version: 3.8.7 (default, Jan 25 2021, 11:14:52) 
[GCC 5.5.0 20171010]
* pandas version: 1.4.0
* sqlite3 version: 2.6.0


## The Movies and Cast Dataset

The data consists of two tables. The first is a table of movies along with (random) audience scores from 1-100. The second is a table of cast members for those movies. There are some interesting cast members in here that you might stumble upon!

Let's read in the database file and show the table descriptions.

In [2]:
disk_engine = db.connect(get_data_path('movieDB.db'))
c = disk_engine.cursor()

c.execute('SELECT type, name, sql FROM sqlite_master')
results = c.fetchall()
for table in results:
    print(table)

('table', 'movies', 'CREATE TABLE movies (id integer, name text, score integer)')
('table', 'cast', 'CREATE TABLE cast (movie_id integer, cast_id integer, cast_name text)')


In [3]:
movies = pd.read_table(get_data_path('movie-name-score.txt'), sep=',', header=None, names=['id', 'name', 'score'])
cast = pd.read_table(get_data_path('movie-cast.txt'), sep=',', header=None, names=['movie_id', 'cast_id', 'cast_name'])

print('Movies Dataframe:')
print('-------------------')
display(movies.head())
print('\n\n')
print('Cast Dataframe:')
print('-------------------')
display(cast.head())

Movies Dataframe:
-------------------


,id,name,score
0,9,Star Wars: Episode III - Revenge of the Sith 3D,61
1,24214,"The Chronicles of Narnia: The Lion, The Witch ...",46
2,1789,War of the Worlds,94
3,10009,Star Wars: Episode II - Attack of the Clones 3D,28
4,771238285,Warm Bodies,3





Cast Dataframe:
-------------------


,movie_id,cast_id,cast_name
0,9,162652153,Hayden Christensen
1,9,162652152,Ewan McGregor
2,9,418638213,Kenny Baker
3,9,548155708,Graeme Blundell
4,9,358317901,Jeremy Bulloch


In terms of Database structures, the **`cast`** table's **`movie_id`** column is a foreign key to the **`movie`** table's **`id`** column. 

This means you can perform any SQL joins or Pandas merges between the two tables on this column. 

One final code cell to get you started - implement the all-too-familiar `canonicalize_tibble` and `tibbles_are_equivalent` functions. 

In [4]:
def canonicalize_tibble(X):
    var_names = sorted(X.columns)
    Y = X[var_names].copy()
    Y.sort_values(by=var_names, inplace=True)
    Y.reset_index(drop=True, inplace=True)
    return Y

def tibbles_are_equivalent (A, B):
    A_canonical = canonicalize_tibble(A)
    B_canonical = canonicalize_tibble(B)
    equal = (A_canonical == B_canonical)
    return equal.all().all()

Let's start with two warm-up exercises. 

**Exercise 0** (2 points): Create a dataframe, ***`cast_size`***, that contains the number of distinct cast members per movie. Your table will have two columns, *`movie_name`*, the name of each film, and *`cast_count`*, the number of unique cast members for the film. 

Order the result by *`cast_count`* from highest to lowest.

> _Note:_ In SQL, the word `cast` is actually a [reserved keyword](https://sqlite.org/lang_expr.html#castexpr), which is used to convert values from one type to another (e.g., a floating-point value to an integer). To distinguish the table named `cast` from the keyword, use the syntax, `[cast]` in SQLite when referencing the table. (This notation is [not standard](https://www3.sqlite.org/lang_keywords.html), and therefore might not port to other SQL implementations.

In [5]:
###
cast_merge = pd.merge(movies, cast, left_on ='id', right_on = 'movie_id', how = 'inner')
cast = cast_merge.rename(columns = {'name': 'movie_name'})
# Count the number of distinct cast members for each movie
cast_size_df = cast.groupby('movie_name')['cast_name'].nunique().reset_index(name='cast_count').sort_values(by='cast_count', ascending=False)

# Display the result
display(cast_size_df)
###


,movie_name,cast_count
102,In Love and War,96
272,The War of the Worlds,78
220,"The Chronicles of Narnia: The Lion, The Witch ...",66
196,Star Wars: Episode III - Revenge of the Sith 3D,66
199,Star Wars: Episode VI - Return of the Jedi,65
...,...,...
44,Crucible of Empire: The Spanish-American War,1
101,Imperial War Museum: The Royal Air Force at War,1
282,They Drew Fire: Combat Artists of World War II,1
186,Secrets of War - Bio & Nuclear Warfare,1


In [10]:
# Use pandas to read a table into a DataFrame
cast_size = pd.read_sql_query('''SELECT DISTINCT m.name AS movie_name, COUNT(DISTINCT c.cast_id) AS cast_count 
                             FROM movies m JOIN [cast] c ON c.movie_id = m.id GROUP BY c.movie_id ORDER BY cast_count DESC''', disk_engine)

# Display the DataFrame
display(cast_size)

,movie_name,cast_count
0,The War of the Worlds,72
1,"The Chronicles of Narnia: The Lion, The Witch ...",66
2,Star Wars: Episode III - Revenge of the Sith 3D,66
3,Star Wars: Episode VI - Return of the Jedi,65
4,This Means War,64
...,...,...
388,Japan's War in Colour,1
389,War of the Century - When Hitler Fought Stalin,1
390,World War 1 in Color,1
391,War Gods of the Deep,1


In [11]:
# Test cell : `test_cast_size`

print("Reading instructor's solution...")

cast_size_solution = pd.read_csv(get_data_path('cast_size_solution.csv'))
display(cast_size_solution[cast_size_solution['movie_name']== 'In Love and War'])
print("Checking...")

# assert set(cast_size.columns) == {'movie_name', 'cast_count'}
# assert tibbles_are_equivalent(cast_size, cast_size_solution), "Your Dataframe is incorrect"
# assert all(cast_size['cast_count'] == cast_size_solution['cast_count'])


# print("\n(Passed!.)")

# del cast_size_solution

Reading instructor's solution...


,movie_name,cast_count
9,In Love and War,56
59,In Love and War,23
170,In Love and War,10
225,In Love and War,7


Checking...


In [13]:
# Test cell : `test_cast_size`

print("Reading instructor's solution...")

cast_size_solution = pd.read_csv(get_data_path('cast_size_solution.csv'))
display(cast_size_solution)
print("Checking...")

assert set(cast_size.columns) == {'movie_name', 'cast_count'}
assert tibbles_are_equivalent(cast_size, cast_size_solution), "Your Dataframe is incorrect"
assert all(cast_size['cast_count'] == cast_size_solution['cast_count'])


print("\n(Passed!.)")

del cast_size_solution

Reading instructor's solution...


,movie_name,cast_count
0,The War of the Worlds,72
1,Star Wars: Episode III - Revenge of the Sith 3D,66
2,"The Chronicles of Narnia: The Lion, The Witch ...",66
3,Star Wars: Episode VI - Return of the Jedi,65
4,This Means War,64
...,...,...
388,Diamonds of War: Africa's Blood Diamond,1
389,War - Greatest Hits Live,1
390,Imperial War Museum: The Royal Air Force at War,1
391,Visions of War: The Algerian War,1


Checking...

(Passed!.)


**Exercise 1** (2 point): Create a dataframe, **`cast_score`**, that contains the average movie score for each cast member. Your table will have two columns, *`cast_name`*, the name of each cast member, and *`avg_score`*, the average movie review score for each movie that the cast member appears in. 

Order this result by `avg_score` from highest to lowest, and round your result for `avg_score` to two (2) decimal places. 

Break any ties in your sorting by cast name in alphabetical order from A-Z. 

In [14]:
cast_score = pd.read_sql_query('''SELECT c.cast_name, ROUND(AVG(m.score),2) AS avg_score 
                  FROM movies m INNER JOIN cast c ON m.id = c.movie_id GROUP BY 1 ORDER BY 2 DESC, 1 ASC''', disk_engine)
display(cast_score)

,cast_name,avg_score
0,Alison Lohman,100.0
1,Andrew Lauer,100.0
2,Chris Sarandon,100.0
3,Clare Holman,100.0
4,Jake Busey,100.0
...,...,...
4692,Timo Torikka,1.0
4693,Tomi Salmela,1.0
4694,Vesa Makela,1.0
4695,Vesa Vierikko,1.0


In [15]:
# Test cell : `test_cast_score`
print("Reading instructor's solution...")

cast_score_solution = pd.read_csv(get_data_path('cast_score_solution.csv'))
display(cast_score_solution)
# print("Checking...")

# assert set(cast_score.columns) == {'cast_name', 'avg_score'}
# assert tibbles_are_equivalent(cast_score, cast_score_solution), "Your Dataframe is incorrect"
# assert all(cast_score['avg_score'] == cast_score_solution['avg_score'])


# print("\n(Passed!)")

# del cast_score_solution

Reading instructor's solution...


,cast_name,avg_score
0,Alison Lohman,100.0
1,Andrew Lauer,100.0
2,Chris Sarandon,100.0
3,Clare Holman,100.0
4,Jake Busey,100.0
...,...,...
4692,Timo Torikka,1.0
4693,Tomi Salmela,1.0
4694,Vesa Makela,1.0
4695,Vesa Vierikko,1.0


In [16]:
# Test cell : `test_cast_score`
print("Reading instructor's solution...")

cast_score_solution = pd.read_csv(get_data_path('cast_score_solution.csv'))

print("Checking...")

assert set(cast_score.columns) == {'cast_name', 'avg_score'}
assert tibbles_are_equivalent(cast_score, cast_score_solution), "Your Dataframe is incorrect"
assert all(cast_score['avg_score'] == cast_score_solution['avg_score'])


print("\n(Passed!)")

del cast_score_solution

Reading instructor's solution...
Checking...

(Passed!)


**Exercise 2** (3 points): You will now create a dataframe, **`one_hit_wonders`**, that contains actors and actresses that appear in **exactly** one movie, with a movie score == 100. Your result will have three columns, *`cast_name`,* the name of each cast member that meets the criteria, *`movie_name`*, the name of the movie that cast member appears in, and *`movie_score`*, which for the purposes of this Exercise is always == 100. 

Order your result by `cast_name` in alphabetical order from A-Z.

In [17]:
display(pd.read_sql_query('''SELECT 
    c.cast_name, 
    m.name, 
    m.score,
    COUNT(c.cast_name) OVER (PARTITION BY c.cast_name) AS cast_count
FROM 
    movies m 
    INNER JOIN cast c ON m.id = c.movie_id
''', disk_engine))

,cast_name,name,score,cast_count
0,A.J. Brown,Unconstitutional: The War on Our Civil Liberties,12,1
1,A.J. Draven,The Thirst: Blood War,66,1
2,Aarno Sulkanen,Talvisota (Winter War),1,1
3,Aaron Au,War (Rogue Assassin),42,1
4,Aaron Hill,A Warrior's Heart,61,1
...,...,...,...,...
5085,Zosia Mamet,War Wolves,73,1
5086,Zsa Zsa Gabor,A Nightmare on Elm Street 3 - Dream Warriors,61,1
5087,Zsuzsa Palos,Hanna's War,50,1
5088,Zura Ingorokva,5 Days Of War,51,1


In [18]:
one_hit_wonders = pd.read_sql_query('''SELECT 
    cast_name, 
    name AS movie_name, 
    score AS movie_score
FROM 
   (SELECT 
    c.cast_name, 
    m.name, 
    m.score,
    COUNT(c.cast_name) OVER (PARTITION BY c.cast_name) AS cast_count
FROM 
    movies m 
    INNER JOIN cast c ON m.id = c.movie_id)
    WHERE cast_count = 1 AND movie_score = 100 ORDER BY cast_name ASC
''', disk_engine)
display(one_hit_wonders)

,cast_name,movie_name,movie_score
0,Alison Lohman,Kaze no tani no Naushika (Nausicaa of the Vall...,100
1,Andrew Lauer,Hg Wells War of the Worlds,100
2,Chris Sarandon,Kaze no tani no Naushika (Nausicaa of the Vall...,100
3,Clare Holman,Island at War,100
4,Jake Busey,Hg Wells War of the Worlds,100
5,James Wilby,Island at War,100
6,Joanne Froggatt,Island at War,100
7,Julia Ford,Island at War,100
8,Patrick Stewart,Kaze no tani no Naushika (Nausicaa of the Vall...,100
9,Peter Greene,Hg Wells War of the Worlds,100


In [19]:
# Test cell : `one_hit_wonders_score`

print("Reading instructor's solution...")

one_hit_wonders_solution = pd.read_csv(get_data_path('one_hit_wonders_solution.csv'))
display(one_hit_wonders_solution)

print("Checking...")

assert set(one_hit_wonders.columns) == {'cast_name','movie_name', 'movie_score'}
assert tibbles_are_equivalent(one_hit_wonders, one_hit_wonders_solution)
assert all(one_hit_wonders['movie_score'] == one_hit_wonders_solution['movie_score'])

print("\n(Passed!)")

del one_hit_wonders_solution

Reading instructor's solution...


,cast_name,movie_name,movie_score
0,Alison Lohman,Kaze no tani no Naushika (Nausicaa of the Vall...,100
1,Andrew Lauer,Hg Wells War of the Worlds,100
2,Chris Sarandon,Kaze no tani no Naushika (Nausicaa of the Vall...,100
3,Clare Holman,Island at War,100
4,Jake Busey,Hg Wells War of the Worlds,100
5,James Wilby,Island at War,100
6,Joanne Froggatt,Island at War,100
7,Julia Ford,Island at War,100
8,Patrick Stewart,Kaze no tani no Naushika (Nausicaa of the Vall...,100
9,Peter Greene,Hg Wells War of the Worlds,100


Checking...

(Passed!)


In [20]:
# Test cell : `one_hit_wonders_score`

print("Reading instructor's solution...")

one_hit_wonders_solution = pd.read_csv(get_data_path('one_hit_wonders_solution.csv'))

print("Checking...")

assert set(one_hit_wonders.columns) == {'cast_name','movie_name', 'movie_score'}
assert tibbles_are_equivalent(one_hit_wonders, one_hit_wonders_solution)
assert all(one_hit_wonders['movie_score'] == one_hit_wonders_solution['movie_score'])

print("\n(Passed!)")

del one_hit_wonders_solution

Reading instructor's solution...
Checking...

(Passed!)


**Exercise 3** (3 points): For this problem, you will find cast members that work well together. We define this as two cast members being in **>= 3** movies together, with the **average movie score being >= 50**. 

You will create a dataframe called **`good_teamwork`** that contains four columns:
- *`cast_member_1`* and *`cast_member_2`*, the names of each pair of cast members that appear in the same movie;
- *`num_movies`*, the number of movies that each pair of cast members appears in; and
- *`avg_score`*, the average review score for each of those movies containing the two cast members. 

Order the results by `cast_member_1` alphabetically from A-Z, and break any ties by sorting by `cast_member_2` alphabetically from A-Z. Round the result for `avg_score` to two (2) decimal places.

One more wrinkle: your solution will likely create several duplicate pairs of cast members: rows such as:

cast_member_1     |cast_member_2  |num_movies  |avg_score
------------------|---------------|------------|---------
 Anthony Daniels  |Frank Oz       |5           |50.60
 Frank Oz         |Anthony Daniels|5           |50.60
 
Remove all duplicate pairs, keeping all cases where `cast_member_1`'s name comes before `cast_member_2`'s name in the alphabet. In the example above, you will keep **only** the first row in your final solution. Make sure to also remove matches where `cast_member_1` == `cast_member_2`.

In [44]:
### df.groupby('col2')['col1'].agg(list).reset_index()
good_pairs = cast.groupby('movie_id')['cast_id'].agg(list).reset_index()
display(good_pairs)
###


,movie_id,cast_id
0,9,"[162652153, 162652152, 418638213, 548155708, 3..."
1,1789,"[162652763, 162652764, 162652765, 162652766, 1..."
2,7583,"[162653923, 162653925, 162653926, 162653927, 1..."
3,9622,"[162657036, 162657037, 162657039, 348469510, 1..."
4,10008,"[162652152, 162652242, 162652154, 162662392, 4..."
...,...,...
388,771266110,"[770922712, 246239042, 771412606, 771412607, 7..."
389,771303871,"[771398698, 771398696, 771398700, 771398699, 7..."
390,771305013,"[771102610, 771374303, 770784362, 771405117, 7..."
391,771311338,"[162659335, 359855197, 770698005, 330424856, 7..."


In [ ]:
# Test cell : `good_teamwork_score`
print("Reading instructor's solution...")

good_teamwork_solution = pd.read_csv(get_data_path('good_teamwork_solution.csv'))
print(good_teamwork_solution)

print("Checking...")

assert set(good_teamwork.columns) == {'cast_member_1','cast_member_2', 'num_movies', 'avg_score'}
assert tibbles_are_equivalent(good_teamwork, good_teamwork_solution)
assert all(good_teamwork['num_movies'] == good_teamwork_solution['num_movies'])
assert all(good_teamwork['avg_score'] == good_teamwork_solution['avg_score'])

print("\n(Passed!)")

del good_teamwork_solution

In [ ]:
c.close()
disk_engine.close()

**Fin!** Remember to test your solutions by running them as the autograder will: restart the kernel and run all cells from "top-to-bottom." Also remember to submit to the autograder; otherwise, you will **not** get credit for your hard work!